In [31]:
library("tidyverse")

In [ ]:
apex factor \
 --out /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Factor_and_Covariate/AC.APEX \
 --iter 1000 \
 --cov "/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov" \
 --factors 4 \
 --bed /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Phenotype/AC.mol_phe.bed.gz

In [ ]:
apex factor \
 --out /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Factor_and_Covariate/AC.APEX \
 --iter 1000 \
 --cov "/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov" \
 --factors 4 \
 --bed /mnt/mfs/statgen/xqtl_workflow_testing/testing_complete/Data_Processing/Phenotype/AC.mol_phe.bed.gz

In [32]:
library("igraph")

In [68]:
kin0 <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.kin0', header=F, stringsAsFactor=F)
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")

In [69]:
# Remove related individuals while keeping maximum number of individuals
# this function is simplified from:
# https://rdrr.io/cran/plinkQC/src/R/utils.R
#' @param relatedness [data.frame] containing pair-wise relatedness estimates
#' (in column [relatednessRelatedness]) for individual 1 (in column
#' [relatednessID1] and individual 2 (in column [relatednessID1]). Columns
#' relatednessID1, relatednessID2 and relatednessRelatedness have to present,
#' while additional columns such as family IDs can be present. Default column
#' names correspond to column names in output of plink --genome
#' (\url{https://www.cog-genomics.org/plink/1.9/ibd}). All original
#' columns for pair-wise highIBDTh fails will be returned in fail_IBD.
#' @param relatednessTh [double] Threshold for filtering related individuals.
#' Individuals, whose pair-wise relatedness estimates are greater than this
#' threshold are considered related.
relatednessFilter <- function(relatedness,
                              relatednessTh,
                              relatednessID1="ID1",
                              relatednessID2="ID2",
                              relatednessRelatedness="KINSHIP") {
    # format data
    if (!(relatednessID1 %in% names(relatedness))) {
        stop(paste("Column", relatednessID1, "for relatedness not found!"))
    }
    if (!(relatednessID2 %in% names(relatedness))) {
        stop(paste("Column", relatednessID1, "for relatedness not found!"))
    }
    if (!(relatednessRelatedness %in% names(relatedness))) {
        stop(paste("Column", relatednessRelatedness,
                   "for relatedness not found!"))
    }

    ID1_index <- which(colnames(relatedness) == relatednessID1)
    ID2_index <- which(colnames(relatedness) == relatednessID2)

    relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
    relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

    relatedness_names <- names(relatedness)
    names(relatedness)[ID1_index] <- "ID1"
    names(relatedness)[ID2_index] <- "ID2"
    names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

    # Remove symmetric IID rows
    relatedness_original <- relatedness
    relatedness <- dplyr::select(relatedness, ID1, ID2, M)

    sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
        c(sort(c(pair[1], pair[2])))
        })), stringsAsFactors=FALSE)
    keepIndex <- which(!duplicated(sortedIDs))

    relatedness_original <- relatedness_original[keepIndex,]
    relatedness <- relatedness[keepIndex,]

    # individuals with at least one pair-wise comparison > relatednessTh
    # return NULL to failIDs if no one fails the relatedness check
    highRelated <- dplyr::filter(relatedness, M > relatednessTh)
    if (nrow(highRelated) == 0) {
        return(list(relatednessFails=NULL, failIDs=NULL))
    }

    # all samples with related individuals
    allRelated <- c(highRelated$ID1, highRelated$ID2)
    uniqueIIDs <- unique(allRelated)

    # Further selection of samples with relatives in cohort
    multipleRelative <- unique(allRelated[duplicated(allRelated)])
    singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

    highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                        highRelated$ID2 %in% multipleRelative,]
    highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                       highRelated$ID2 %in% singleRelative,]

    # Only one related samples per individual
    if(length(singleRelative) != 0) {
      # randomly choose one to exclude
      failIDs_single <- highRelatedSingle[,1]

    } else {
      failIDs_single <- NULL
    }

    # An individual has multiple relatives
    if(length(multipleRelative) != 0) {
        relatedPerID <- lapply(multipleRelative, function(x) {
            tmp <- highRelatedMultiple[rowSums(
                cbind(highRelatedMultiple$ID1 %in% x,
                      highRelatedMultiple$ID2 %in% x)) != 0,1:2]
            rel <- unique(unlist(tmp))
            return(rel)
        })
        names(relatedPerID) <- multipleRelative

        keepIDs_multiple <- lapply(relatedPerID, function(x) {
            pairwise <- t(combn(x, 2))
            index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                          highRelatedMultiple$ID2 %in% pairwise[,2]) |
                (highRelatedMultiple$ID1 %in% pairwise[,2] &
                     highRelatedMultiple$ID2 %in% pairwise[,1])
            combination <- highRelatedMultiple[index,]
            combination_graph <- igraph::graph_from_data_frame(combination,
                                                               directed=FALSE)
            all_iv_set <- igraph::ivs(combination_graph)
            length_iv_set <- sapply(all_iv_set, function(x) length(x))

            if (all(length_iv_set == 1)) {
                # check how often they occurr elsewhere
                occurrence <- sapply(x, function(id) {
                    sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                })
                # if occurrence the same everywhere, pick the first, else keep
                # the one with minimum occurrence elsewhere
                if (length(unique(occurrence)) == 1) {
                    nonRelated <- sort(x)[1]
                } else {
                    nonRelated <- names(occurrence)[which.min(occurrence)]
                }
            } else {
                nonRelated <- all_iv_set[which.max(length_iv_set)]
            }
            return(nonRelated)
        })
        keepIDs_multiple <- unique(unlist(keepIDs_multiple))
        failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                   keepIDs_multiple])
    } else {
        failIDs_multiple <- NULL
    }
    allFailIIDs <- c(failIDs_single, failIDs_multiple)
    relatednessFails <- lapply(allFailIIDs, function(id) {
        fail_inorder <- relatedness_original$ID1 == id &
            relatedness_original$M > relatednessTh
        fail_inreverse <- relatedness_original$ID2 == id &
            relatedness_original$M > relatednessTh
        if (any(fail_inreverse)) {
            inreverse <- relatedness_original[fail_inreverse, ]
            id1 <- ID1_index
            id2 <- ID2_index
            inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
            names(inreverse) <- relatedness_names
        } else {
            inreverse <- NULL
        }
        inorder <- relatedness_original[fail_inorder, ]
        names(inorder) <- relatedness_names
        return(rbind(inorder, inreverse))
    })
    relatednessFails <- do.call(rbind, relatednessFails)
    if (nrow(relatednessFails) == 0) {
        relatednessFails <- NULL
        failIDs <- NULL
    } else {
        names(relatednessFails) <- relatedness_names
        rownames(relatednessFails) <- 1:nrow(relatednessFails)
        uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
        failIDs <- uniqueFails[,ID1_index]
    }
    return(list(relatednessFails=relatednessFails, failIDs=failIDs))
}


# main code


[1] 1


In [67]:
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")

In [ ]:
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
if (TRUE) {
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")$failIDs
    tmp1 <- kin0[,1:2]
    tmp2 <- kin0[,3:4]
    colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
    # Get the family ID for these rels so there are two columns FID and IID in the output
    lookup <- dplyr::distinct(rbind(tmp1,tmp2))
    dat <- lookup[which(lookup[,2] %in% rel),]
} else {
    rel <- kin0 %>% filter(KINSHIP >= 0.0625)
    IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
    dat <- data.frame(IID)
    dat <- dat %>%
        mutate(FID = IID) %>%
        select(FID, IID)
}
cat("There are", nrow(dat),"related individuals using a kinship threshold of 0.0625\n")
write.table(dat,'/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.tmp.related_id', quote=FALSE, row.names=FALSE, col.names=FALSE)

In [61]:
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
if (TRUE) {
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")$failIDs
    tmp1 <- kin0[,1:2]
    tmp2 <- kin0[,3:4]
    colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
    # Get the family ID for these rels so there are two columns FID and IID in the output
    lookup <- dplyr::distinct(rbind(tmp1,tmp2))
    dat <- lookup[which(lookup[,2] %in% rel),]
} 

In [63]:
relatednessFilter <- function(relatedness, 
                                  relatednessTh,
                                  relatednessID1="ID1", 
                                  relatednessID2="ID2",
                                  relatednessRelatedness="KINSHIP") {
        # format data
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

        relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"
        print(1)
        # Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1, ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]
        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }

        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]
        print(2)
        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
        print(3)
        # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }
        print(4)
        return(list(relatednessFails=relatednessFails, failIDs=failIDs))
    }
    

In [81]:
        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
        print(3)
       
    

[1] 3


In [84]:
if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })

ERROR: Error in parse(text = x, srcfile = src): <text>:9:0: unexpected end of input
7:                 return(rel)
8:             })
  ^


In [86]:
      relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })

In [87]:
relatedPerID

[[1]]
 [1] "SM-CJEG8"    "SM-CJEIH"    "SM-CJEJO"    "SM-CJFNF"    "SM-CJGIE"   
 [6] "SM-CJGMZ"    "SM-CJGNB"    "SM-CJIXD"    "SM-CJIXS"    "SM-CJIYB"   
[11] "SM-CJIYE"    "SM-CJJ34"    "SM-CJK53"    "SM-CTDV9"    "SM-CTEDD"   
[16] "SM-CTEGM"    "SM-CTEM1"    "SM-CTEMX"    "MAP50103679" "ROS20998065"

[[2]]
[1] "SM-CJEHD"    "SM-CTEDD"    "ROS20376029" "SM-CJEFR"   

[[3]]
 [1] "SM-CJFKV"    "SM-CJFM7"    "SM-CJFMX"    "SM-CJFOL"    "SM-CJFP3"   
 [6] "SM-CJGHI"    "SM-CJGIE"    "SM-CJGLZ"    "SM-CJGM2"    "SM-CJGMZ"   
[11] "SM-CJGNB"    "SM-CJIXL"    "SM-CJIXT"    "SM-CJIY4"    "SM-CJJ1E"   
[16] "SM-CJJ1H"    "SM-CJJ1P"    "SM-CJJ1Z"    "SM-CJJ2A"    "SM-CJK3Y"   
[21] "SM-CTDRZ"    "SM-CTDSK"    "SM-CTDTG"    "SM-CTDTZ"    "SM-CTDUE"   
[26] "SM-CTDUG"    "SM-CTDUX"    "SM-CTDVA"    "SM-CTDVF"    "SM-CTEDD"   
[31] "SM-CTEE6"    "SM-CTEFW"    "SM-CTEMR"    "SM-CTEMS"    "SM-CTEMW"   
[36] "MAP50103679" "ROS79590778" "SM-CJEG7"    "SM-CJEJ4"   

[[4]]
[1] "SM-CJFM7"    "SM-CJFOA"    "SM-CJGHS"    "SM-CJIY3"    "SM-CTDRL"   
[6] "SM-CTDUF"    "MAP50103679" "SM-CJEG7"    "SM-CJFKV"   

[[5]]
 [1] "SM-CJFMD"    "SM-CJFOL"    "SM-CJGH4"    "SM-CJGHS"    "SM-CJGIE"   
 [6] "SM-CJGMG"    "SM-CJGMZ"    "SM-CJGNH"    "SM-CJIWW"    "SM-CJJ2X"   
[11] "SM-CJK4Y"    "SM-CTEI8"    "MAP50103679" "SM-CJEJO"   

[[6]]
[1] "SM-CJFMJ"    "SM-CJIZD"    "SM-CJK5B"    "MAP15387421" "SM-CJEI7"   

[[7]]
 [1] "SM-CJFMX" "SM-CJFP3" "SM-CJIY4" "SM-CJK5J" "SM-CJK46" "SM-CJK53"
 [7] "SM-CTEFD" "SM-CTEFW" "SM-CJEKW" "SM-CJFKV" "SM-CJFLF"

[[8]]
[1] "SM-CJFN3" "SM-CJFO6" "SM-CJFO7" "SM-CJIY8" "SM-CTEI7" "SM-CJEI7" "SM-CJEJ2"
[8] "SM-CJEKW"

[[9]]
 [1] "SM-CJFNF" "SM-CJFO6" "SM-CJFOE" "SM-CJGNH" "SM-CJJ2E" "SM-CJJ2X"
 [7] "SM-CTECW" "SM-CTEDD" "SM-CTEE4" "SM-CTEF7" "SM-CTEGI" "SM-CTEI7"
[13] "SM-CTEMR" "SM-CJEFS" "SM-CJEG8" "SM-CJEJ2"

[[10]]
 [1] "SM-CJFNP"    "SM-CJGLF"    "SM-CJGMG"    "SM-CJGNH"    "SM-CJIXS"   
 [6] "SM-CJK3K"    "SM-CJK5H"    "SM-CTDTK"    "SM-CTDTZ"    "SM-CTEI7"   
[11] "SM-CTEMS"    "MAP50301099" "SM-CJEH2"    "SM-CJEI7"    "SM-CJEIH"   
[16] "SM-CJEJ2"   

[[11]]
[1] "SM-CJFO6" "SM-CJFOA" "SM-CJGGL" "SM-CJGMG" "SM-CTDRR" "SM-CTEFJ" "SM-CTEN3"
[8] "SM-CJFN3" "SM-CJFNF"

[[12]]
[1] "SM-CJFOA" "SM-CJJ1E" "SM-CTEFW" "SM-CTEMT" "SM-CJEK5" "SM-CJFM7" "SM-CJFMU"
[8] "SM-CJFO6"

[[13]]
[1] "SM-CJFOE"    "MAP34726040" "SM-CJFNF"   

[[14]]
 [1] "SM-CJFOL" "SM-CJFP3" "SM-CJGGQ" "SM-CJGIE" "SM-CJGMG" "SM-CJIY4"
 [7] "SM-CJJ1H" "SM-CJJ2G" "SM-CJJ2H" "SM-CTDTG" "SM-CTDVA" "SM-CTENF"
[13] "SM-CJEJ2" "SM-CJFKE" "SM-CJFKV" "SM-CJFMD"

[[15]]
 [1] "SM-CJFP3"    "SM-CJGHS"    "SM-CJGIE"    "SM-CJGM2"    "SM-CJGNH"   
 [6] "SM-CJIXL"    "SM-CTDT9"    "SM-CTEMS"    "MAP50103679" "MAP50301099"
[11] "SM-CJEIE"    "SM-CJFKV"    "SM-CJFMX"    "SM-CJFOL"   

[[16]]
[1] "SM-CJFP5"    "SM-CJGMG"    "SM-CJIXS"    "SM-CJIZD"    "SM-CJIZO"   
[6] "SM-CJK3K"    "ROS20376029" "SM-CJEFR"    "SM-CJEKW"   

[[17]]
 [1] "SM-CJGH4" "SM-CJGNI" "SM-CJIY1" "SM-CJJ2G" "SM-CJJ2H" "SM-CJK3K"
 [7] "SM-CTDRL" "SM-CTDRT" "SM-CJEHN" "SM-CJEIL" "SM-CJFMD"

[[18]]
 [1] "SM-CJGHI"    "SM-CJGI1"    "SM-CJGNB"    "SM-CJIXS"    "SM-CJJ1E"   
 [6] "SM-CJK4R"    "SM-CTDT9"    "SM-CTDTI"    "SM-CTDU5"    "SM-CTEI7"   
[11] "MAP46246604" "ROS20376029" "SM-CJEFR"    "SM-CJEGX"    "SM-CJFKV"   
[16] "SM-CJFO5"    "SM-CJFOR"   

[[19]]
 [1] "SM-CJGHS"    "SM-CJGIE"    "SM-CJGM2"    "SM-CJGMG"    "SM-CJGMZ"   
 [6] "SM-CJGN1"    "SM-CJIWW"    "SM-CJIXS"    "SM-CJIY3"    "SM-CJIYE"   
[11] "SM-CJJ2G"    "SM-CJJ2X"    "SM-CJK3K"    "SM-CJK44"    "SM-CTDRY"   
[16] "SM-CTDT1"    "SM-CTDT9"    "SM-CTDUX"    "SM-CTDV6"    "SM-CTDV9"   
[21] "SM-CTEDD"    "SM-CTEE6"    "SM-CTEF7"    "SM-CTEGB"    "SM-CTEI7"   
[26] "SM-CTEM1"    "SM-CTEMX"    "MAP50409406" "SM-CJEGG"    "SM-CJEKI"   
[31] "SM-CJFLL"    "SM-CJFLR"    "SM-CJFM1"    "SM-CJFM7"    "SM-CJFMD"   
[36] "SM-CJFNK"    "SM-CJFO7"    "SM-CJFP3"    "SM-CJGGX"   

[[20]]
 [1] "SM-CJGIE" "SM-CJJ1H" "SM-CTDS7" "SM-CTDUX" "SM-CTEF7" "SM

In [90]:
 keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))})

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })

In [91]:
  lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))})

In [93]:
test = function(x){
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))}

In [97]:
                pairwise <- t(combn(x, 2))

In [99]:
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])

In [103]:
                combination <- highRelatedMultiple[index,]


In [105]:
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)

In [107]:
all_iv_set <- igraph::ivs(combination_graph)

In [116]:
length(all_iv_set[[6]])

[1] 1

In [95]:
x = relatedPerID[[1]]

In [83]:
 # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }


In [51]:
        rel <- relatednessFilter(kin0, ${kinship}, "ID1", "ID2", "KINSHIP")$failIDs
        tmp1 <- kin0[,1:2]
        tmp2 <- kin0[,3:4]
        colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
        # Get the family ID for these rels so there are two columns FID and IID in the output
        lookup <- dplyr::distinct(rbind(tmp1,tmp2))
        dat <- lookup[which(lookup[,2] %in% rel),]
    } else {
        rel <- kin0 %>% filter(KINSHIP >= ${kinship})
        IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
        dat <- data.frame(IID)
        dat <- dat %>%
            mutate(FID = IID) %>%
            select(FID, IID)
    }
    cat("There are", nrow(dat),"related individuals using a kinship threshold of ${kinship}\n")
    write.table(dat,${_output:r}, quote=FALSE, row.names=FALSE, col.names=FALSE)

There are 0 related individuals using a kinship threshold of ${kinship}


In [56]:
    write.table(dat,"~/empty_related", quote=FALSE, row.names=FALSE, col.names=FALSE)

In [70]:
relatedness = kin0
                            relatednessTh = 0.0625
                                  relatednessID1="ID1"  
                                  relatednessID2="ID2" 
                                  relatednessRelatedness="KINSHIP"

In [71]:
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

In [72]:
 relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

In [73]:
# Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1,ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]

In [75]:
length(keepIndex)

[1] 631

In [76]:

        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }


In [119]:
if(F){}

In [78]:


        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]


In [14]:
# Remove symmetric IID rows
        relatedness_original <- relatedness


In [13]:
relatedness <- dplyr::select(relatedness, ID1,ID2,M)

In [29]:
 
        # format data
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

        relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

        # Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1,ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]
        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }

        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]

        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
  
        # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }

$relatednessFails
NULL

$failIDs
NULL

ERROR: Error in if (nrow(relatednessFails) == 0) {: argument is of length zero


In [30]:
relatednessFails


NULL

In [22]:
relatednessRelatedness

[1] "KINSHIP"

In [23]:
relatedness

FID1,ID1,FID2,ID2,NSNP,HETHET,IBS0,M
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
ROS20998065,ROS20998065,MAP50103679,MAP50103679,260130,0.0984700,3.28605e-02,0.0628394
ROS79590778,ROS79590778,ROS20376029,ROS20376029,260353,0.0962386,3.17185e-02,0.0625368
SM-CJEFR,SM-CJEFR,ROS20376029,ROS20376029,260480,0.2451900,7.29423e-05,0.4940130
SM-CJEFZ,SM-CJEFZ,MAP50402431,MAP50402431,260442,0.2333960,8.44718e-05,0.4934330
SM-CJEG8,SM-CJEG8,MAP50103679,MAP50103679,260243,0.1007370,3.41489e-02,0.0635570
SM-CJEG8,SM-CJEG8,ROS20998065,ROS20998065,260146,0.1035800,3.34082e-02,0.0707883
SM-CJEGZ,SM-CJEGZ,ROS10524640,ROS10524640,260325,0.2473020,6.53030e-05,0.4952050
SM-CJEHD,SM-CJEHD,ROS20376029,ROS20376029,260295,0.0948078,2.88711e-02,0.0703430
SM-CJEHD,SM-CJEHD,SM-CJEFR,SM-CJEFR,260336,0.0946162,2.89357e-02,0.0697255


In [ ]:
# Load required libraries
library(dplyr)
library(flashpcaR)
# Read the PLINK binary files
frel <- '/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.unrelated'
# Read loadings, center and scale from previous PCA
dat <- readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.exp.pca.rds')
f <- dat$pca_model
# compute centroids before projecting back the samples
# (calculate mean/median/cov per pop)
if(TRUE){
    pop_group <- split(dat$pc_scores[ ,c(paste0("PC_", 1:5))], list(Group = dat$pc_scores$RACE))
    dat$pc_cov <- lapply(pop_group, function(x) cov(x))
    dat$pc_mean <- lapply(pop_group, function(x) sapply(x, mean))
    dat$pc_median <- lapply(pop_group, function(x) sapply(x, median))
} else {
    dat$pc_cov <- cov(f$projection[,1:5])
    dat$pc_mean <- apply(f$projection[,1:5], 2, mean)
    dat$pc_median <- apply(f$projection[,1:5], 2, median)
}

# Read the bim file to obtain reference alleles
bim <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.unrelated.bim',stringsAsFactors =F)
ref <- as.character(bim[,5])
names(ref) <- bim[,2]
overlapped_variants <- match(names(ref), rownames(f$loadings))
# Project related samples
fpro <- project(frel, loadings=f$loadings[overlapped_variants,], orig_mean=f$center[overlapped_variants], orig_sd=f$scale[overlapped_variants], ref_allele=ref)
pca <- fpro$projection
k = 10
pca <- as.data.frame(fpro$projection)
pca <- tibble::rownames_to_column(pca, "ID")
colnames(pca) <- c("ID",paste0("PC_", 1:k))
pca$IID <- sapply(strsplit(as.character(pca$ID),':'), "[", 1)
# Read fam file with phenotypes
pheno <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.exp', sep="\t", header=T, stringsAsFactors =F)
pca <-merge(pheno, pca, by="IID", all=FALSE)
dat$pc_scores = rbind(dat$pc_scores, pca)

# Write the PC scores to a file
write.table(dat$pc_scores,"/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.AC.pca.projected.txt", sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
# save results
saveRDS(dat, '/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.AC.pca.projected.rds')